In [102]:
from dotenv import load_dotenv
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langchain_core.messages import AIMessage, HumanMessage, BaseMessage, AIMessageChunk
from langchain_community.chat_models import ChatZhipuAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import os
from typing import Sequence
from typing_extensions import Annotated, TypedDict
from langgraph.graph.message import add_messages
from typing import Dict, Any

In [91]:
class State(TypedDict):
    input: str
    chat_history: Annotated[Sequence[BaseMessage], add_messages]
    answer: str
    metadata: Dict[str,Any]

In [92]:
load_dotenv()
ZHIPU_API_KEY = os.getenv("ZHIPU_API_KEY")
llm = ChatZhipuAI(
    api_key=ZHIPU_API_KEY,
    temperature=0.5,
    model="glm-4-flash",
    streaming=True,
    stream_usage=True
)

In [93]:
system_prompt = "你是一个为了软件架构而生的AI助手，辅助进行软件架构设计，你的名字是HIT软件架构小助手"
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [104]:
chain = prompt | llm


def call_model(state: State):

    response = chain.invoke(state)
    return {
        "chat_history": [
            HumanMessage(state["input"]),
            AIMessage(response.content),
        ],
        "answer": response.content,
        "metadata": response.response_metadata,
    }


# Define a new graph
workflow = StateGraph(state_schema=State)

# Define the (single) node in the graph
workflow.add_edge(START, "llm")
# workflow.add_node("llm", call_model)


In [105]:
def add_history(state: State):
    state["chat_history"].append(HumanMessage("hello"))
    
# workflow.add_node("history_update", add_history)
workflow.add_node("llm", call_model)
# workflow.add_edge("history_update", "llm")
# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [111]:
config = {"configurable": {"thread_id": "aa22"}}
query = "你好，我叫Bob"
def start():
    for msg,metadata in app.stream({'input':'你好'},config=config,stream_mode='messages'):
        if isinstance(msg, AIMessageChunk):
            yield msg.content
print(start())

<generator object start at 0x116e7d540>


In [1]:
from dbutils.pooled_db import PooledDB
import pymysql
import json
connection_pool = PooledDB(
    creator=pymysql,
    db="sharding_db",
    user="root",
    password="root",
    host="192.168.2.8",
    port=3321,
    mincached=2,  # 最小空闲连接数
    maxcached=5,  # 最大空闲连接数
    maxconnections=None,  # 最大连接数
)
conn = connection_pool.connection()
cursor = conn.cursor()

In [4]:
message = {
    "conversation_id":"3590f90f-f04b-4388-8bde-cea5b4720b4a",
    "title":"hello",
    "chat_history":[
        {"role": "user", "content": "Hello!"},
        {"role": "assistant", "content": "Hi, how can I help you?"}
    ],
    "tokens_used": 5
}
conversation_id = message.get("conversation_id", "")
tokens_used = message.get("tokens_used", 0)
chat_history = message.get("chat_history", "")
chat_history = json.dumps(chat_history)
title = message.get("title", "")

In [5]:
cursor.execute(
    """
    INSERT INTO t_chat_history (conversation_id, title, chat_history, token_usage)
    VALUES (%s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE
        title = VALUES(title),
        chat_history = VALUES(chat_history),
        token_usage = VALUES(token_usage)
    """,
    (conversation_id, title, chat_history, tokens_used),
)
conn.commit()